# Subsystem Restart

This demonstrates "subsystem restart" feature of platform management framework,
which allows restarting a user-defined subsystem without affecting any other subsystem.

**NOTE:** For Petalinux BSP v2020.1, Platform Management Framework only supports "default subsystem"
which includes Arm Cortex-A72 (APU) and Cortex-R5F (RPU) processors.
Since this demo restarts the default subsystem which includes APU and RPU,
in effect linux running on APU will be restarted.
As a result, jupyter notebook server process will be killed,
and in turn this notebook session will have to be restarted on reboot.

1. [Introduction](#introduction)
2. [Implementation details](#implementation-details)
3. [Subsystem-Restart Demo](#subsys-restart-demo)
4. [References](#xlnx-pm-wiki)

---

## Introduction <a name="introduction"></a>

Versal supports various types of reset - from the simplest system reset to the much more complicated subsystem restart.

In any system or subsystem which has a processor component and a programmable logic component, reset must entail both reset to the hardware as well as software.

Reset to the hardware includes resetting of the processor, all peripherals associated with the system/subsystem,
cleaning up of the memory as needed, and making sure that the interconnect is in a clean state, capable of routing traffic.

Reset to the software results in the processor starting from the reset vector.
However, designer must make sure that valid and clean code for system/subsystem is located at the reset vector in order to bring the system back to a clean running state.

The intent of the "subsystem restart" feature is to demonstrate how a design with multiple subsystems can cleanly and reliably restart a subsystem without affecting other subsystems in the design. 
A subsystem is defined as combination of one or more Master and Slave components that work together to accomplish a function. 
A subsystem can be all the components of a particular operating system, including both PS and PL, but it could also be a function without an operating system, such as PL or ME.

---

## Implementation details <a name="implementation-details"></a>

If you're not using Petalinux pre-built images and compiling using your own config,
the kernel option `ZYNQMP_FIRMWARE` and `ZYNQMP_FIRMWARE_DEBUG` needs to be enabled while compiling the kernel in order to enable this feature and its debugfs interface.

To trigger the subsystem restart, we can use the standard linux `reboot` interface; however we need to change the scope of the reset from `system` which is default to `subsystem`.

Lets see the available scopes:

In [ ]:
cat /sys/firmware/zynqmp/shutdown_scope

 - `system`: Sets the scope to system. Reboot/shutdown will affect the whole system.
 - `subsystem`: Sets the scope to subsystem. Reboot/shutdown will reset the subsystem associated with the calling master. Components that are not part of the calling subsystem will not be affected.
 - `ps_only`: For Versal, this is not supported since the definition of PS is defined by the user.
 
 To change the scope, simply write the string `"subsystem"` to `/sys/firmware/zynqmp/shutdown_scope`.

---

## Subsystem Restart Demo <a name="subsys-restart-demo"></a>

To run this demo, following requirements **must** be met:
  1. Boot image must be packaged with a special format which supports multiple images.
  2. System must be booted from a Non-JTAG boot mode.

For BSP release v2020.1, the default boot image format file (BIF) does not support multiple images packaged in one boot image;
which means that the boot image from the BSP (`BOOT.BIN`) won't work for the purpose of this demo.

For this reason, we have provided a boot image (`boot-demo.bin`) which can be used instead of the one present in petalinux BSP pre-builts.
For the users who wish to use this demo using their designs, a boot image format file (`boot-demo.bif`) is also provided which can be used with `bootgen` tool to generate a boot image with the env variable `BOOTGEN_SUBSYSTEM_PDI=1`. Please refer to the SW developers guide for more details on how to package boot image.

The artifacts for this demo can be located at: `/usr/share/example-notebooks/pm-notebooks/images/`

In [ ]:
!ls -la /usr/share/example-notebooks/pm-notebooks/images/

Now that you have booted using `boot-demo.bin`, lets run the demo:

**NOTE:** This will restart linux!

Set the shutdown scope to `subsystem` and issue `reboot`.

In [ ]:
!echo subsystem > /sys/firmware/zynqmp/shutdown_scope && reboot

## References <a name="xlnx-pm-wiki"></a>
 - [Zynq UltraScale+ MPSoC Restart solution](https://xilinx-wiki.atlassian.net/wiki/spaces/A/pages/18841820/Zynq+UltraScale+MPSoC+Restart+solution)